In [1]:
import nest_asyncio
nest_asyncio.apply()

from playwright.async_api import async_playwright
import asyncio
import re

URL = "https://sadmoody.github.io/unwordle/"

In [4]:
async def main():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        await page.goto(URL, wait_until="networkidle")
        await page.wait_for_timeout(2000)
        
        # hard coding hints rn
        hints = [
            {
                "word": "slate",
                "breakdown": [('s','X'), ('l','X'), ('a','X'), ('t','~'), ('e','X')]
            },
            {
                "word": "tight",
                "breakdown": [('t','O'), ('i','X'), ('g','X'), ('h','X'), ('t','X')]
            }
        ]

        # helper to index tiles by row/col
        def tile_at(row, col):
            # row, col are 0-based
            return page.locator("game-tile").nth(row * 5 + col)

        for row_index, hint in enumerate(hints):
            # dict access, not attribute access
            await page.keyboard.type(hint["word"])
            await page.keyboard.press("Enter")
            await page.wait_for_timeout(1000)

            # ('a','O'), ('p','O'), ...
            for col_index, (letter, status) in enumerate(hint["breakdown"]):
                tile_el = tile_at(row_index, col_index)

                # pattern-match on the status char, NOT the whole tuple
                match status:
                    case 'O':
                        clicks = 2      # gray -> yellow -> green
                    case '~':
                        clicks = 1      # gray -> yellow
                    case 'X':
                        clicks = 0      # leave as gray
                    case _:
                        raise ValueError("improper hint status")

                # click this tile the required number of times
                for _ in range(clicks):
                    await tile_el.click()
                    await page.wait_for_timeout(200)

        # get the suggestions
        first_suggestion = page.locator("div.suggestion").nth(1) #0th is the title
        await first_suggestion.wait_for(state="attached", timeout=5000)
        raw = await first_suggestion.inner_text()
        try:
            _, word = raw.split(".", 1)
            word = word.strip()
        except ValueError:
            raise ValueError(f"Suggestion parsing failed; got: {raw!r}")

        await browser.close()

        print(f"suggested word: {word}")
        return word

await main()


suggested word: TRUNK


'TRUNK'